This is the open source code of paper: Utilizing Domain Knowledge: Robust Machine Learning for Building Energy Prediction with Small, Inconsistent Datasets.
- @Date : 2022-06-24 16:21:35
- @Link : https://arxiv.org/abs/2302.10784
- @Ver : v02
- @Author: Xia CHEN (xia.chen@iek.uni-hannover.de), Xia Chen, Manav Mahan Singh, Philipp Geyer
For using the code or data, please cite:

*- Chen, X., Singh, M.M. and Geyer, P., 2023. Utilizing Domain Knowledge: Robust Machine Learning for Building Energy Prediction with Small, Inconsistent Datasets. arXiv preprint arXiv:2302.10784.*

*- CHEN, Xia; Singh, Manav Mahan; Geyer, Philipp (2023), “Utilizing domain knowledge: robust machine learning for building energy performance prediction with small, inconsistent datasets”, Mendeley Data, V2, doi: 10.17632/fctghwx3r9.2*

---

This code is designed for implementing the core of Component-based Machine Learning (CBML) framework, by using previous component and zone level ML model for predicting building level heat flows and their impact on the building's heating load. Available ML models are: `DecisionTree`, `SGDRegressor`, `LightGBM`. They are involved in proving that the effectiveness of CBML framework is model-independent. The process is divided into several distinct stages:

---
1. **Data Preparation**: The code starts by preparing the data necessary for the prediction process. It reads a CSV file containing the building zones' typical day characteristics for a given period (`df_route`). The dataframe is initially augmented with an identifier column (`'id'`) that combines file names and zone names to uniquely identify each entry.
---
2. **Feature Selection and Model Prediction**: The script defines a set of input features relevant to the building's thermal characteristics and external conditions, including month, day, hour, temperature, humidity, and various building design parameters like wall area and window-to-wall ratio (WWR). These features are used to predict the heat flows through different components (roof, wall/window, infiltration, ground floor) using pretrained LightGBM models.
---
3. **Component Heat Flow Calculation**: For each building component (roof, wall/window, infiltration, ground floor), a separate CSV file with predictions is read. The predictions are processed to calculate the total heat flow from each component, taking into account the number of occurrences of each file in the dataset to properly replicate the data and ensure consistency in the aggregation process.
---
4. **Aggregation and Final Prediction**: After calculating the component heat flows, these values are merged with the original dataset. This enriched dataset now serves as input to another ML model to predict the overall heating load of the building. This stage signifies the integration of component-level predictions into a holistic assessment of the building's thermal performance.
---
5. **Performance Evaluation**: Finally, the predicted heating load is compared against actual values to evaluate the model's performance using various metrics (RMSE, SMAPE, MAE, MSE, R^2). This evaluation provides insights into the accuracy and reliability of the model predictions.
---
6. **Date Handling and Result Output**: The script also handles date information by combining year, month, day, and hour into a single datetime object. This datetime information, alongside the predicted and actual heating load values, can be used for further analysis or visualizations.
---

In [1]:
# Building performance simulation, component

import pandas as pd
import numpy as np
import os, sys, gc, time, warnings, pickle, psutil, random
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn import metrics
import csv
from math import ceil

from tqdm import tqdm

import plotly.offline as py                    
py.init_notebook_mode(connected=True)          
import plotly.graph_objs as go                 
import plotly.figure_factory as ff             
import plotly.io as pio                        

# Target plot
import seaborn as sns, matplotlib.pyplot as plt
import random 

# model training
import lightgbm as lgb
from sklearn.model_selection import train_test_split


# pd.set_option('max_columns', 50)

warnings.filterwarnings('ignore')
ROUTE = '../_TrainData_done/'
ROUTE_test = '../_TestData_done/'

g = os.walk(ROUTE)  

files = []
for path,dir_list,file_list in g:  
    for file_name in file_list:  
        files.append(file_name)


PERIOD = files[0].split('_')[1][:-4]
files

['BuildingTypicalDays_Winter Typical.csv',
 'GFloorTypicalDays_Winter Typical.csv',
 'InfiltrationTypicalDays_Winter Typical.csv',
 'RoofTypicalDays_Winter Typical.csv',
 'WallWindowTypicalDays_Winter Typical.csv',
 'ZoneTypicalDays_Winter Typical.csv']

In [2]:
########################### Helpers #################################################################################
## Seeder
# :seed to make all processes deterministic     # type: int
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

    
## Multiprocess Runs
def df_parallelize_run(func, t_split):
    num_cores = np.min([N_CORES,len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

# :df pandas dataframe to reduce size             # type: pd.DataFrame()
# :verbose                                        # type: bool
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# 创建文件夹
def mkdir(path):
    path=path.strip()
    path=path.rstrip("\\")
    isExists=os.path.exists(path)
    if not isExists:
        os.makedirs(path) 
        
# Result & Evaluation
def RMSE(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())


def SMAPE(F, A):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))


def MAE(F, A):
    return 1/len(A) * np.sum(np.abs(F - A))


def MSE(F, A):
    return 1/len(A) * np.sum(pow(np.abs(F - A),2))

# check the value distribution of training output, it's important for hyperparameter tuning
def plot_target(df, TARGET):
    plt.figure(figsize=(20,12))
    plot_df = df[TARGET]
    # his_grid = random.sample(list(plot_df), 10000)

    sns.distplot(plot_df,label=TARGET)
    plt.legend(loc='upper right')
    plt.show()
    print('Done!')

In [3]:
########################### Model params #################################################################################

def train_test_df(component, df, df_test, lgb_params, features, TARGET):  
    # spilt Train/Test
    X_train = df[features]
    y_train = df[TARGET]

    X_test = df_test[features]
    y_test = df_test[TARGET]
    
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

#     # Test on different ML models
#     scaler = StandardScaler()  
#     X_train = scaler.fit_transform(X_train)
#     X_test = scaler.transform(X_test)
#     # save the scaler
#     regr = tree.DecisionTreeRegressor()
#     regr.fit(X_train, y_train)
    
#     y_pred = regr.predict(X_test)
#     print('DecisionTree',r2_score(y_test, y_pred))

#     regr= linear_model.SGDRegressor()
#     regr.fit(X_train, y_train)
#     y_pred = regr.predict(X_test)
#     print('SGDRegressor', r2_score(y_test, y_pred))

    train_data = lgb.Dataset(X_train, 
                       label=y_train)
    # train_data.save_binary('train_data.bin')
    # train_data = lgb.Dataset('train_data.bin')

    # valid_data = lgb.Dataset(vaild_df[features], 
    #                    label=vaild_df[Target_features[0]])

    estimator = lgb.cv(
                        lgb_params,
                        train_data,
                        num_boost_round=10000,
                        nfold=3,
                        early_stopping_rounds=100,
                        verbose_eval=100,
                        stratified=False,
                        seed=42
    )
    rounds = int(len(estimator['rmse-mean']))
    print('The best RMSE in CV is {:.5f}，std {:.5f}.'.format(
    estimator['rmse-mean'][-1], estimator['rmse-stdv'][-1]))

    print('The best iteration round is {}.'.format(len(estimator['rmse-mean'])))
    
    estimator = lgb.train(lgb_params,
                          train_data,
                          verbose_eval = 100,
                          num_boost_round = int(len(estimator['rmse-mean'])), ###########
                          )
    ############################ Making result df ############################
    test_df = pd.DataFrame(y_test)
    test_df['pred'] = estimator.predict(X_test)

    ############################ Result evaluation ###########################
    msg = 'Result RMSE is {}, SMAPE is {}, MAE is {}, MSE is {}, R^2 is {}'.format('%.4f' % RMSE(test_df['pred'],test_df[TARGET]), 
                                                             '%.4f' % SMAPE(test_df['pred'],test_df[TARGET]), 
                                                             '%.4f' % MAE(test_df['pred'],test_df[TARGET]),
                                                             '%.4f' % MSE(test_df['pred'],test_df[TARGET]),
                                                             '%.4f' % metrics.r2_score(test_df[TARGET],test_df['pred']))
    print(msg)
    ############################ Make dir ###########################
    mkdir(component + '_' + TARGET)
    
    ########################### Feature importance ##########################
    plt.rcParams["figure.figsize"] = (10,10)
    # mean decrease impurity
    imp_img = lgb.plot_importance(estimator, ignore_zero=False)
    imp_img.figure.savefig(component + '_' + TARGET + '/' + TARGET + '_feature_imp.png')
#     imp_img.save_img(TARGET + '/' + TARGET + '_feature_imp.png')
    ########################### save model ##############################
    estimator.save_model(component + '_' + TARGET + '/' + TARGET + '_model.txt')
#     #load from model:
#     bst = lgb.Booster(model_file='mode.txt')
    ########################### log result ##############################
    with open("result.csv","a+", newline='') as csvfile: 
        writer = csv.writer(csvfile)
        writer.writerows([[component, TARGET, str(rounds), str(RMSE(test_df['pred'],test_df[TARGET])), str(SMAPE(test_df['pred'],test_df[TARGET])),str(MAE(test_df['pred'],test_df[TARGET])), str(MSE(test_df['pred'],test_df[TARGET])), str(metrics.r2_score(test_df[TARGET],test_df['pred'])), len(features)]])
        
    return

In [4]:
################################### component prediction ###############################################
def load_component_predict(model_route, features, df_route, num_dict, TARGET, REPEAT=False):
#     bst_roof = lgb.Booster(model_file='model/Heat Flow_GFloor.txt')
#     bst_infi = lgb.Booster(model_file='model/Heat Flow_Infiltration.txt')
#     bst_wallwin = lgb.Booster(model_file='model/Heat Flow_WallWindow.txt')
#     bst = lgb.Booster(model_file='model/Heat Flow_Roof.txt')
#     X = df_test['Area', 'U Value', 'Heat Capacity']
#     df_test['pred'] = bst.predict(X)
    df = pd.read_csv(df_route)
#     df = df.set_index('File',drop=True)
    df.head()
    
    bst = lgb.Booster(model_file=model_route)
    X = df[features]
    df['pred'] = bst.predict(X)
    df['id'] = df['File'] + df['Zone Name']
    
    if(REPEAT is True):
        new_df = pd.DataFrame()
        for a,b in zip(num_dict.index.tolist(), num_dict['File']):
            b = int(b/312)
            for i in range(0,b):
                new_df = pd.concat([new_df,df[df['File'] == a]],axis=0)
    else:
        new_df = df

    new_df = new_df[['id', TARGET + ':Heat Flow','pred','File']]
    
    return new_df


In [5]:
df_route = ROUTE_test + 'ZoneTypicalDays_' + PERIOD + '.csv'
print(df_route)
df = pd.read_csv(df_route)
df['id'] = df['File'] + df['Name']
df.head()

Input_features = ['Month', 'Day', 'Hour', 'Temp', 'Dew', 'Hum', 'Pres', 
                 'id', 'Internal Wall Area', 'Internal Floor Area', 'Heat Capacity (Floor Slabs)', 
                 'Internal Mass', 'WWR (North)', 'WWR (East)', 'WWR (West)', 'WWR (South)', 
                 'Building_Start Time', 'Building_Operating Hours', 'Building_Light Heat Gain', 
                 'Building_Equipment Heat Gain', 'Building_Occupancy', 'Building_Heating Setpoint', 'Building_Cooling Setpoint','Wall Heat Flow', 'Window Heat Flow']

num_dict = pd.DataFrame(df['File'].value_counts())
num_dict = num_dict.loc[df['File'].drop_duplicates().tolist()]
num_dict

../_TestData_done/ZoneTypicalDays_Winter Typical.csv


,File
Shape101_17,1560
Shape101_0,1560
Shape101_16,936
Shape101_5,1248
Shape101_18,1560
...,...
Shape101_192,936
Shape101_195,1248
Shape101_183,1248
Shape101_191,936


In [6]:
df.shape

(1248000, 51)

In [7]:
PERIOD

'Winter Typical'

In [8]:
# Merge with weather data

weather_df = pd.read_csv('../Weather_data.csv')
# to date_time and set as index
date_df = weather_df[['Year', 'Month', 'Day', 'Hour']]
date = pd.to_datetime(date_df)

weather_df.index = date

del date_df, date

weather_df = weather_df[['Year', 'Month', 'Day', 'Hour','Temp','Dew','Hum','Pres']]
if(PERIOD == 'Winter Typical'):
    # 1/5-1/17
    target_weather = weather_df.loc[(weather_df['Month'] == 1)&(weather_df['Day'] >= 5)&(weather_df['Day'] <= 17)]
if(PERIOD == 'Winter Extreme'):
    # 2/9-2/21
    target_weather = weather_df.loc[(weather_df['Month'] == 2)&(weather_df['Day'] >= 9)&(weather_df['Day'] <= 21)]
if(PERIOD == 'Spring'):
    # 3/28-4/10
    target_weather = weather_df.loc[(weather_df['Month'] == 2)&(weather_df['Day'] >= 9)&(weather_df['Day'] <= 21)]
if(PERIOD == 'Summer Typical'):
    # 7/12 - 7/24
    target_weather = weather_df.loc[(weather_df['Month'] == 7)&(weather_df['Day'] >= 12)&(weather_df['Day'] <= 24)]
if(PERIOD == 'Summer Extreme'):
    # 7/19-7/31
    target_weather = weather_df.loc[(weather_df['Month'] == 7)&(weather_df['Day'] >= 19)&(weather_df['Day'] <= 31)]
if(PERIOD == 'Autumn'):
    # 10/19-10/31
    target_weather = weather_df.loc[(weather_df['Month'] == 10)&(weather_df['Day'] >= 19)&(weather_df['Day'] <= 31)]
target_weather

,Year,Month,Day,Hour,Temp,Dew,Hum,Pres
1986-01-05 01:00:00,1986,1,5,1,-5.4,-7.7,82,95700
1986-01-05 02:00:00,1986,1,5,2,-6.9,-8.4,88,95700
1986-01-05 03:00:00,1986,1,5,3,-6.6,-7.9,89,95700
1986-01-05 04:00:00,1986,1,5,4,-8.2,-10.7,80,95600
1986-01-05 05:00:00,1986,1,5,5,-7.9,-10.2,82,95600
...,...,...,...,...,...,...,...,...
1986-01-17 20:00:00,1986,1,17,20,-2.1,-2.7,95,96300
1986-01-17 21:00:00,1986,1,17,21,-1.9,-2.9,92,96400
1986-01-17 22:00:00,1986,1,17,22,-4.2,-5.0,93,96500
1986-01-17 23:00:00,1986,1,17,23,-4.9,-5.5,95,96600


In [9]:
# component prediction

# features = ['Month','Day','Hour','Temp','Dew','Hum',
#             'Pres', 'Area', 'U Value', 'Heat Capacity']

TARGET =  PERIOD + '_Heat Flow'
result_route = files[3][:-4] + '_' + TARGET + '/' + TARGET + '.csv'
df_roof = pd.read_csv(result_route)

df_roof = df_roof[['File', 'Zone Name', 'Name', 'pred']]
df_roof.columns = ['File', 'Zone Name', 'Name', 'Roof Heat Flow']

new_df = pd.DataFrame()
for a,b in zip(num_dict.index.tolist(), num_dict['File']):
    b = int(b/312)
    for i in range(0,b):
        new_df = pd.concat([new_df,df_roof[df_roof['File'] == a]],axis=0)
df_roof = new_df


print(df_roof.shape)
print('Roof Done!')

######################################

result_route = files[4][:-4] + '_' + TARGET + '/' + TARGET + '.csv'
df_ww = pd.read_csv(result_route)

df_ww = df_ww[['File', 'Zone Name', 'Name', 'pred']]
df_ww.columns = ['File', 'Zone Name', 'Name', 'WallWindow Heat Flow']
print(df_ww.shape)
# merge target_weather
target_weather = target_weather.reset_index(drop=True)
repeat_time = int(len(df_ww)/len(target_weather))

merge_weather = pd.concat([target_weather]*repeat_time, ignore_index=True)
# output_df = output_df.reset_index(drop=True)
df_ww = pd.concat([merge_weather, df_ww], axis=1)
df_ww['id'] = df_ww['File'] + df_ww['Zone Name'] + df_ww['Month'].values.astype(str).tolist() + df_ww['Day'].values.astype(str).tolist() + df_ww['Hour'].values.astype(str).tolist()
df_ww['WallWindow Heat Flow'] = df_ww.groupby(by='id')['WallWindow Heat Flow'].transform('sum')
df_ww = df_ww[['WallWindow Heat Flow','id']]
df_ww = df_ww.drop_duplicates(subset=['id'])

df_ww = df_ww.reset_index()
print(df_ww.shape)
print('WallWindow Done!')


result_route = files[2][:-4] + '_' + TARGET + '/' + TARGET + '.csv'
df_inf = pd.read_csv(result_route)

df_inf = df_inf[['File', 'Zone Name', 'Name', 'pred']]
df_inf.columns = ['File', 'Zone Name', 'Name', 'Infiltration Heat Flow']
print(df_inf.shape)
print('Infiltration Done!')


result_route = files[1][:-4] + '_' + TARGET + '/' + TARGET + '.csv'
df_gfloor = pd.read_csv(result_route)

df_gfloor = df_gfloor[['File', 'Zone Name', 'Name', 'pred']]
df_gfloor.columns = ['File', 'Zone Name', 'Name', 'GFloor Heat Flow']

new_df = pd.DataFrame()
for a,b in zip(num_dict.index.tolist(), num_dict['File']):
    b = int(b/312)
    for i in range(0,b):
        new_df = pd.concat([new_df,df_gfloor[df_gfloor['File'] == a]],axis=0)
df_gfloor = new_df


print(df_gfloor.shape)
print('GFloor Done!')

(6240000, 4)
Roof Done!
(10608000, 4)
(1248000, 3)
WallWindow Done!
(1248000, 4)
Infiltration Done!
(1248000, 4)
GFloor Done!


In [10]:
result_route = files[5][:-4] + '_' + TARGET + '/' + TARGET + '_model.txt'
result_route

'ZoneTypicalDays_Winter Typical_Winter Typical_Heat Flow/Winter Typical_Heat Flow_model.txt'

In [ ]:
############################# merge component heat flow ############
X = df[Input_features]
X = pd.concat([X,df_roof['Roof Heat Flow'].reset_index()], axis=1)
X = pd.concat([X,df_ww['WallWindow Heat Flow'].reset_index()], axis=1)
X = pd.concat([X,df_inf['Infiltration Heat Flow'].reset_index()], axis=1)
X = pd.concat([X,df_gfloor['GFloor Heat Flow'].reset_index()], axis=1)
X = X.fillna(0)

########################### load zone area & predict ##############
features = ['Month', 'Day', 'Hour', 'Temp', 'Dew', 'Hum', 'Pres', 
             'Internal Wall Area', 'Internal Floor Area', 'Heat Capacity (Floor Slabs)', 
             'Internal Mass', 'WWR (North)', 'WWR (East)', 'WWR (West)', 'WWR (South)', 
             'Building_Start Time', 'Building_Operating Hours', 'Building_Light Heat Gain', 
             'Building_Equipment Heat Gain', 'Building_Occupancy', 'Building_Heating Setpoint', 'Building_Cooling Setpoint',
             'Wall Heat Flow', 'Window Heat Flow', 'WallWindow Heat Flow', 'GFloor Heat Flow', 'Roof Heat Flow', 'Infiltration Heat Flow']


X = X[features]
X = X.round(3)
# route_list = ['Heating Load','Cooling Load','Lights Load']
route_list = [PERIOD+':Heating Load']
# load model
for each_route in route_list:
#     model_route = 'model/Heating Load_Zone.txt'
#     model_route = 'model/'+ each_route + '_Zone.txt'
    TARGET = PERIOD + '_Heating Load'
    result_route = files[5][:-4] + '_' + TARGET + '/' + TARGET + '_model.txt'
    bst = lgb.Booster(model_file=result_route)

    # predict
    df['pred'] = bst.predict(X)
    # ['WallWindow Heat Flow', 'GFloor Heat Flow', 'Roof Heat Flow', 'Infiltration Heat Flow']

    test_df = df[[each_route, 'pred']]
    TARGET = each_route

    msg = 'Result RMSE is {}, SMAPE is {}, MAE is {}, MSE is {}, R^2 is {}'.format('%.4f' % RMSE(test_df['pred'],df[TARGET]), 
                                                             '%.4f' % SMAPE(test_df['pred'],df[TARGET]), 
                                                             '%.4f' % MAE(test_df['pred'],df[TARGET]),
                                                             '%.4f' % MSE(test_df['pred'],df[TARGET]),
                                                             '%.4f' % metrics.r2_score(df[TARGET],test_df['pred']))
    print(msg)
    
    date_df = df[['Year', 'Month', 'Day', 'Hour']]
    date = pd.to_datetime(date_df)
    test_df['Date'] = date